
2. O seguinte sistema dinâmico denota 4 inversores ($$\,A, B, C, D\,$$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.
![](/static/img/pixel.gif)

![](https://paper-attachments.dropboxusercontent.com/s_9896551CC5FAD2B2EB6E4EBC08522545FA66314D29FE6A5BE8E593259F8E8A37_1669554332522_inversores.png)

        1. Cada inversor tem um bit $$s$$ de estado, inicializado  com um valor aleatório.
        2. Cada inversor é regido pelas seguintes transformações
                       $\mathbf{invert}\mathtt(in,out)$
                          $x \gets \mathsf{read}(\mathtt{in})$ 
                          $s \gets \neg x\;\;\|\;\; s\gets s\oplus x$               
                       $\mathsf{write}(\mathtt{out},s)$
        iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
        iii. O estado do sistema é um duplo definido pelos 4 bits $$s$$, e é inicializado com um vetor aleatório em $$\{0,1\}^4\;$$.
        iv. O sistema termina em ERRO quando o estado do sistema for $$\,(0,0,0,0)\,$$.
                    
    1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $$\,n\,$$ estados.
    2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

In [43]:
import random as rd
from pysmt.shortcuts import *
from pysmt.typing import INT

In [44]:
def error(state):
    return And(Equals(state['a'],Int(0)),Equals(state['b'], Int(0)), Equals(state['c'], Int(0)), Equals(state['d'], Int(0)))


In [45]:
def declare(i):
    state = {}
    
    # definir variáveis
    state['a'] = Symbol('a' + str(i),INT)
    state['b'] = Symbol('b' + str(i),INT)
    state['c'] = Symbol('c' + str(i),INT)
    state['d'] = Symbol('d' + str(i),INT)
    
    return state

In [46]:
def init(state):
    return And(
        Equals(state['a'], Int(rd.randint(0,1))),
        Equals(state['b'], Int(rd.randint(0,1))),
        Equals(state['c'], Int(rd.randint(0,1))),
        Equals(state['d'], Int(rd.randint(0,1)))
    )

In [53]:
def trans(curr, prox, erro):
        #transicao de a para b
        t0=And(Not(Equals(curr['a'], curr['c'])), Not(Equals(prox['b'],curr['a'])) )
        #transicao de b para d
        t1=And(Not(Equals(curr['b'], curr['a'])), Not(Equals(prox['d'],curr['b'])) )
        #transicao de d para c
        t2=And(Not(Equals(curr['d'], curr['b'])), Not(Equals(prox['d'],curr['c'])) )
        #transicao de c para a
        t3=And(Not(Equals(curr['c'], curr['d'])), Not(Equals(prox['a'],curr['c'])) )
        #estado de erro
        t4=Not(erro(curr))
        return Or(t0,t1,t2,t3,t4)


In [54]:
def geraTraco(init, declare, trans, erro,k):
    with Solver(name="z3") as s:
        traco = [declare(i) for i in range(k)]
        
        s.add_assertion(init(traco[0]))
        
        for i in range(k-1):
            s.add_assertion(trans(traco[i], traco[i+1],erro))
            
        if s.solve():
            print("Here")
            for v in traco[0]:
                print(v, "=", s.get_value(traco[0][v]))

In [55]:
geraTraco(init, declare, trans, error,5)

Here
a = 0
b = 1
c = 0
d = 1


In [67]:
def bmc(declare, init, trans, k, erro):
    with Solver(name="z3") as s:
        for i in range(1,k+1):
            traco = [declare(i) for i in range(k)]
            s.add_assertion(init(traco[0]))
            
            for i in range(k-1):
                s.add_assertion(trans(traco[i], traco[i+1], erro))
                
            #falta invariante
            
            if s.solve():
                print("Não tem solucinha")
                return 
            
    print(f"Propriedade válida para traços de tamanho até {k}")
    
        

In [79]:
bmc(declare, init, trans, 2, error)

Propriedade válida para traços de tamanho até 2
